### Demo to Apply the House Price Prediction Model 

In [ ]:
# A) Importing Libraries  

import pandas as pd
import joblib
import torch
from torch import nn
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [ ]:
# Define the original model architecture used during training to defines the structure of the neural network, including the types of layers
class HousePriceModel(nn.Module):
    def __init__(self, input_dim):
        super(HousePriceModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, 512)  # Changed neurons
        self.bn1 = nn.BatchNorm1d(512)  # Added Batch Normalization
        self.dropout1 = nn.Dropout(0.3)  # Changed dropout
        self.fc2 = nn.Linear(512, 256)
        self.bn2 = nn.BatchNorm1d(256)
        self.dropout2 = nn.Dropout(0.3)
        self.fc3 = nn.Linear(256, 128)
        self.bn3 = nn.BatchNorm1d(128)
        self.dropout3 = nn.Dropout(0.3)
        self.fc4 = nn.Linear(128, 64)
        self.fc5 = nn.Linear(64, 1)  # We Can Added Another Layer

    def forward(self, x):
        x = torch.relu(self.bn1(self.fc1(x)))
        x = self.dropout1(x)
        x = torch.relu(self.bn2(self.fc2(x)))
        x = self.dropout2(x)
        x = torch.relu(self.bn3(self.fc3(x)))
        x = self.dropout3(x)
        x = torch.relu(self.fc4(x))
        x = self.fc5(x)
        return x


In [ ]:
# Load the saved preprocessor and model
preprocessor = joblib.load('preprocessor.pkl')

# Load the new house data
new_house_data = pd.read_csv('test2.csv')

# Ensure the new house data has the same structure as the training data
X_new_house = new_house_data.drop(columns=['ID'])

# Preprocess the new house data
X_new_house_preprocessed = preprocessor.transform(X_new_house)  # This may return a sparse matrix

# Convert the sparse matrix to a dense matrix
if hasattr(X_new_house_preprocessed, 'toarray'):
    X_new_house_preprocessed = X_new_house_preprocessed.toarray()

# Determine the input dimension
input_dim = X_new_house_preprocessed.shape[1]

# Initialize the model with the correct input dimension
model = HousePriceModel(input_dim)
model.load_state_dict(torch.load('house_price_model.pth'))
model.eval()

# Convert the preprocessed data to a PyTorch tensor
X_new_house_tensor = torch.tensor(X_new_house_preprocessed.astype('float32'))

# Predict the house price for the new house
with torch.no_grad():
    new_house_price_pred = model(X_new_house_tensor).numpy()

# Add the prediction to the new house data
new_house_data['SalePrice'] = new_house_price_pred

# Save the predictions to a CSV file
new_house_data.to_csv('New_House_Predicted_Price1.csv', index=False)

# Display the predicted house price
print(new_house_data[['ID', 'SalePrice']])

# Define the Mean Absolute Percentage Error (MAPE) function
def mean_absolute_percentage_error(y_true, y_pred):
    return torch.mean(torch.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
# If the true sale prices are available in the new data, calculate the accuracy metrics
if 'TrueSalePrice' in new_house_data.columns:
    y_true = new_house_data['TrueSalePrice']
    y_pred = new_house_data['SalePrice']
    
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    
    print(f'Mean Absolute Error (MAE): {mae}')
    print(f'Mean Squared Error (MSE): {mse}')
    print(f'R-squared (R²): {r2}')
    
    # Convert to tensors for MAPE calculation
    y_true_tensor = torch.tensor(y_true.values.astype('float32'))
    y_pred_tensor = torch.tensor(y_pred.values.astype('float32'))
    
    mape = mean_absolute_percentage_error(y_true_tensor, y_pred_tensor)
    accuracy_rate = 100 - mape.item()  # Convert to accuracy percentage
    
    print(f'Accuracy Rate: {accuracy_rate:.2f}%')
else:
    print("True SalePrice values are not available. Unable to calculate accuracy metrics.")